In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# CI/CD for Model Deployment

<table align="left">
  <td>
    <a href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/fraudfinder/raw/main/06_model_training_pipeline.ipynb">
       <img src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Google Cloud Notebooks">Open in Cloud Notebook
    </a>
  </td> 
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/fraudfinder/blob/main/06_model_training_pipeline.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Open in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/fraudfinder/blob/main/06_model_training_pipeline.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

Your CI/CD pipeline deployed in the previous noebooks, continuously delivers new pipeline implementations to the target environment that in turn delivers prediction services of the newly trained model.



### Objective

This notebook shows how to use Cloud Build to deploy a model generated by execution of Vertex AI Pipeline. 

This lab uses the following Google Cloud services and resources:

- [Vertex AI](https://cloud.google.com/vertex-ai/)
- [BigQuery](https://cloud.google.com/bigquery/)

Steps performed in this notebook:

    * Deployment of Model triggered on-demand (from within notebook)
    * Model CI/CD triggered by source change in Github

### Load configuration settings from the setup notebook

Set the constants used in this notebook and load the config settings from the `00_environment_setup.ipynb` notebook.

In [ ]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
BUCKET_NAME = f"{PROJECT_ID}-fraudfinder"
config = !gsutil cat gs://{BUCKET_NAME}/config/notebook_env.py
print(config.n)
exec(config.n)

In [ ]:
PIPELINE_NAME = f'fraud-finder-xgb-pipeline-{ID}'
PIPELINE_STORE_URI = f"gs://{BUCKET_NAME}/pipeline-store/"
IMAGE_REPOSITORY=f"fraudfinder-{ID}"
ENDPOINT_DISPLAY_NAME = f'fraudfinder-endpoint-{ID}'
MODEL_DISPLAY_NAME = f'fraudfinder-xgb-{ID}'

### A) Deployment of Model triggered on-demand (from within notebook)

This CI/CD will deploy the latest model in the Vertex Model. 

TO DO: to add unit_test, evaluation creteria, release note, manual approval, etc into CI/CD

In [ ]:
SUBSTITUTIONS=f"""\
_REGION={REGION},\
_ENDPOINT_DISPLAY_NAME={ENDPOINT_DISPLAY_NAME},\
_MODEL_DISPLAY_NAME={MODEL_DISPLAY_NAME},\
_WORKSPACE_DIR=./\
"""

!echo $SUBSTITUTIONS

In [ ]:
!gcloud builds submit /home/jupyter/fraudfinder/vertex_ai/ \
--timeout=306m \
--machine-type=e2-highcpu-8 \
--config=deploy_model/modeldeployment.yaml \
--substitutions= --substitutions {SUBSTITUTIONS}


### B) Model CI/CD triggered by source change in Github

In [ ]:
# TO DO: to integrate it with Marc's git repo
# !gcloud beta builds triggers create github \
# --name=triger-ff-mlops \
# --repo-name=gcp-demo \
# --repo-owner=saeedaghabozorgi \
# --branch-pattern=main \
# --included-files=mlops-fraudfinder/** \
# --build-config=mlops-fraudfinder/train/notebooks/cloudbuild.yaml \
# --substitutions=_IMAGE_REPOSITORY="fraudfinder-381"